In [119]:
!pip install pyspark

In [167]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("spark").getOrCreate()


In [186]:

accidents=spark.read.csv("/content/Road_accidents.csv",header=True)
accidents_time=spark.read.csv("/content/Road_accidents_time.csv",header=True).toDF("STATE/UT","YEAR","O-3(n)","3-6(n)","6-9(d)","9-12(d)","12-15(d)","15-18(d)","18-21(n)","21-24(n)")
accidents.show()
accidents_time.show()

+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+
|      STATE/UT|YEAR|JANUARY|FEBRUARY|MARCH|APRIL| MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|
+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+
| A & N Islands|2001|      8|      23|   15|   15|  14|  19|  14|    19|        7|     12|      13|      22|
| A & N Islands|2002|     12|      10|   14|   16|  10|   7|  16|    11|       23|     21|      11|      17|
| A & N Islands|2003|     19|      13|   15|   13|  13|  12|   8|    16|       17|     25|      14|      15|
| A & N Islands|2004|     21|      14|   22|   17|  13|  18|  16|    19|       16|     20|      15|      24|
| A & N Islands|2005|     19|      21|   22|   17|  13|  19|  21|    14|       15|     19|      10|      16|
| A & N Islands|2006|     21|      13|    4|   22|   9|  14|  12|    14|        8|     14|       6|      18|
| A & N Islands|200

In [169]:
#1.Add column 'Total', giving sum of accidents in each month
from pyspark.sql.functions import *
list=[month for month in accidents.columns[2:]]
exp='+'.join(list)
accidents = accidents.withColumn("Total", expr(exp))
accidents.show()


+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+-------+
|      STATE/UT|YEAR|JANUARY|FEBRUARY|MARCH|APRIL| MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|  Total|
+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+-------+
| A & N Islands|2001|      8|      23|   15|   15|  14|  19|  14|    19|        7|     12|      13|      22|  181.0|
| A & N Islands|2002|     12|      10|   14|   16|  10|   7|  16|    11|       23|     21|      11|      17|  168.0|
| A & N Islands|2003|     19|      13|   15|   13|  13|  12|   8|    16|       17|     25|      14|      15|  180.0|
| A & N Islands|2004|     21|      14|   22|   17|  13|  18|  16|    19|       16|     20|      15|      24|  215.0|
| A & N Islands|2005|     19|      21|   22|   17|  13|  19|  21|    14|       15|     19|      10|      16|  206.0|
| A & N Islands|2006|     21|      13|    4|   22|   9|  14|  12

In [170]:
#2.Which state has highest number of accidents in year 2013
highest_no_of_accidents=accidents.select("YEAR","State/UT","Total").filter(col("YEAR")==2013).orderBy("Total",ascending=False).first()[1]
highest_no_of_accidents

'Tamil Nadu'

In [171]:
#3.Average monthly accidents for each state
monthly_accidents=accidents.groupBy("STATE/UT").agg(avg(col("JANUARY")).alias("AVG_JAN"),
                                                    avg(col("FEBRUARY")).alias("AVG_FEB"),
                                                    avg(col("MARCH")).alias("AVG_MAR"),
                                                    avg(col("APRIL")).alias("AVG_APR"),
                                                    avg(col("MAY")).alias("AVG_MAY"),
                                                    avg(col("JUNE")).alias("AVG_JUNE"),
                                                    avg(col("JULY")).alias("AVG_JUL"),
                                                    avg(col("AUGUST")).alias("AVG_AUG"),
                                                    avg(col("SEPTEMBER")).alias("AVG_SEP"),
                                                    avg(col("OCTOBER")).alias("AVG_OCT"),
                                                    avg(col("NOVEMBER")).alias("AVG_NOV"),
                                                    avg(col("DECEMBER")).alias("AVG_DEC"))


monthly_accidents.show()


+-----------------+------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+------------------+
|         STATE/UT|           AVG_JAN|            AVG_FEB|           AVG_MAR|            AVG_APR|           AVG_MAY|          AVG_JUNE|            AVG_JUL|           AVG_AUG|            AVG_SEP|            AVG_OCT|            AVG_NOV|           AVG_DEC|
+-----------------+------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+------------------+
|         Nagaland| 5.071428571428571|  5.857142857142857| 5.142857142857143|  4.928571428571429|               4.0| 5.357142857142857|  5.071428571428571| 3.642857142857143| 3.0714285714285716|  5.214285714285714| 3.2857142857142856| 5.1

In [172]:
#4. Which month has highest accidents
highest_acc = accidents.select([col for col in accidents.columns if col!= 'STATE/UT' and col!='YEAR' and col!="Total"])
var  = {}
for i in highest_acc.columns:
  var[i] = highest_acc.agg({i:'sum'}).collect()[0][0]
df4 = spark.createDataFrame(var.items(), ["month", "value"])
maxValue = df4.agg({'value':'max'}).collect()[0][0]
df4.select('month','value').where(col('value')==maxValue).show()


+-----+--------+
|month|   value|
+-----+--------+
|  MAY|521563.0|
+-----+--------+



In [187]:
#5. Which time slot (like 0-3, 3-6 etc) has more accidents? What about 'Karnataka' state?
df5=accidents_time.drop("YEAR")
df5=df5.drop("STATE/UT")
df5.show()
var  = {}
for i in df5.columns:
  var[i] = df5.agg({i:'sum'}).collect()[0][0]
df4 = spark.createDataFrame(var.items(), ["month", "value"])
maxValue = df4.agg({'value':'max'}).collect()[0][0]
df4.select('month','value').where(col('value')==maxValue).show()


+------+------+------+-------+--------+--------+--------+--------+
|O-3(n)|3-6(n)|6-9(d)|9-12(d)|12-15(d)|15-18(d)|18-21(n)|21-24(n)|
+------+------+------+-------+--------+--------+--------+--------+
|     2|     6|    29|     40|      39|      40|      18|       7|
|     2|     6|    22|     41|      33|      33|      23|       8|
|     2|     8|    31|     35|      28|      36|      25|      15|
|     2|     5|    29|     42|      43|      43|      37|      14|
|     0|     8|    27|     28|      38|      42|      50|      13|
|     1|     3|    17|     33|      33|      23|      38|       7|
|     2|     5|    20|     30|      30|      27|      31|       7|
|     3|     7|    33|     24|      40|      31|      40|      13|
|     2|     6|    35|     41|      64|      54|      50|      19|
|     2|    10|    36|     45|      64|      57|      53|      18|
|     4|     3|    36|     27|      46|      50|      51|      18|
|     4|     6|    26|     29|      54|      45|      58|     

In [161]:
#6. Which state has more accidents from year 2001 to 2014?
#col=[month for month in accidents.columns[2:-1]]
filter_by_year=accidents.filter(col("YEAR").between(2001,2014))
total_accidents_per_state=filter_by_year.groupBy("State/UT").agg(sum("Total").alias("Total"))
max_state=total_accidents_per_state.orderBy("Total",ascending=False).first()[0]
print("State with max accidents from 2001 to 2014:",max_state)

State with max accidents from 2001 to 2014: Tamil Nadu


In [138]:
#7. List states whose accidents number in year 2014 is less than state average from 2001 to 2014.
state_average=accidents.filter(col("YEAR").between(2001,2014)).groupBy("State/UT").agg(avg("Total").alias("Avg_Total")).orderBy("State/UT")
filter_by_year=accidents.select(col("State/UT"),col("Total")).filter(col("YEAR")==2014).orderBy("State/UT")
df1=filter_by_year.join(state_average,on="State/UT")
df2=df1.filter(col("Total")<col("Avg_Total"))
df1.show()
df2.select(("State/UT")).show()



+-----------------+-------+------------------+
|         State/UT|  Total|         Avg_Total|
+-----------------+-------+------------------+
|    A & N Islands|  218.0|206.64285714285714|
|   Andhra Pradesh|43232.0|39058.642857142855|
|Arunachal Pradesh|  185.0|242.07142857142858|
|            Assam| 7144.0| 4408.428571428572|
|            Bihar| 9531.0| 6617.714285714285|
|       Chandigarh|  366.0|457.92857142857144|
|     Chhattisgarh|12140.0|            8415.5|
|       D&N Haveli|   87.0|              87.0|
|      Daman & Diu|   39.0| 45.92857142857143|
|         Delhi Ut| 8310.0|            8310.0|
|              Goa| 4230.0|3209.3571428571427|
|          Gujarat|22152.0|21451.785714285714|
|          Haryana|11043.0|  9422.92857142857|
| Himachal Pradesh| 3325.0|2650.3571428571427|
|  Jammu & Kashmir| 5778.0| 5915.714285714285|
|        Jharkhand| 4905.0|3300.6428571428573|
|        Karnataka|43694.0|41768.642857142855|
|           Kerala|35872.0|           37011.5|
|      Laksha

In [256]:
#8. Which month has more accidents over the year?
highest_acc = accidents.select([col for col in accidents.columns if col!= 'STATE/UT' and col!='YEAR'])
var  = {}
for i in highest_acc.columns:
  var[i] = highest_acc.agg({i:'sum'}).collect()[0][0]
df4 = spark.createDataFrame(var.items(), ["month", "value"])
maxValue = df4.agg({'value':'max'}).collect()[0][0]
df4.select('month','value').where(col('value')==maxValue).show()

+-----+--------+
|month|   value|
+-----+--------+
|  MAY|521563.0|
+-----+--------+

